In [ ]:
import re
import  requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import numpy
import itertools
pd.set_option('display.max_row',100)
from tqdm import tqdm

In [ ]:
url = 'https://www.boatrace.jp/owpc/pc/race/racelist?rno=9&jcd=22&hd=20221229'
res = requests.get(url)
res.encoding = res.apparent_encoding
soup = BeautifulSoup(res.text, 'html.parser')

In [ ]:
soup.find('p').text

'\nレーサー検索\n'

## スクレイピング

In [ ]:
df = pd.DataFrame()
for p in tqdm(range(1,31)):
  if p == 1 or p==2 or p==3 or p==4 or p==5 or p==6 or p==7 or p==8 or p==9:
    p = '0' + str(p)
  for r in tqdm(range(1,13)):
    url = 'https://www.boatrace.jp/owpc/pc/race/racelist?rno='+str(r)+'&jcd=22&hd=202212'+str(p)
    result_url = 'https://www.boatrace.jp/owpc/pc/race/raceresult?rno='+str(r)+'&jcd=22&hd=202212'+str(p)

    res = requests.get(url)
    res.encoding = res.apparent_encoding
    soup = BeautifulSoup(res.text, 'html.parser')
    print(soup.find('p').text)

    if soup.find('p').text == '\nレーサー検索\n':
      #名前
      name_list_1 = []
      name_list = []
      name = soup.find_all('div',class_="is-fs18")
      name_list_1.append(name)
      for i in name_list_1[0]:
        name_list.append(i.get_text())

      #年齢　体重
      class_age_weight = soup.find_all('div',class_="is-fs11")
      age_weight = []
      x = 1
      for i in class_age_weight:
        if x % 2 == 0:
          age_weight.append(i.get_text())
        x += 1

      age = []
      weight = []
      for i in age_weight:
        age.append(i[30:33])
        weight.append(i[34:38])

      x = 1
      class_list = []
      for i in class_age_weight:
        if x % 2 == 1:
          i = i.get_text()
          class_list.append(i[35:37])
        x += 1
      #==================
      data_1 = soup.find_all(class_="is-lineH2")

      F_count = []
      L_count = []
      mean_ST = []
      #==================
      win_rate = []
      win2_rate = []
      win3_rate = []
      #=================
      local_win_rate = []
      local_win2_rate = []
      local_win3_rate = []
      #=================
      motor_num = []
      motor_win2 = []
      motor_win3 = []
      #=================
      boat_num = []
      boat_win2 = []
      boat_win3 = []
      x = 0
      for i in range(6):
        F_count.append(data_1[x].get_text()[0:2])
        L_count.append(data_1[x].get_text()[27:29])
        mean_ST.append(data_1[x].get_text()[54:58])
      #==============================================
        win_rate.append(data_1[x+1].get_text()[0:4])
        win2_rate.append(data_1[x+1].get_text()[27:34])
        win3_rate.append(data_1[x+1].get_text()[57:64])
      #==============================================
        local_win_rate.append(data_1[x+2].get_text()[0:4])
        local_win2_rate.append(data_1[x+2].get_text()[29:34])
        local_win3_rate.append(data_1[x+2].get_text()[57:64])
      #==============================================
        motor_num.append(data_1[x+3].get_text()[0:3])
        motor_win2.append(data_1[x+3].get_text()[27:32])
        motor_win3.append(data_1[x+3].get_text()[57:64])
      #===============================================
        boat_num.append(data_1[x+4].get_text()[0:3])
        boat_win2.append(data_1[x+4].get_text()[28:33])
        boat_win3.append(data_1[x+4].get_text()[58:63])
        x += 5

      course = [1,2,3,4,5,6]
      new_df = pd.DataFrame({'名前':name_list,
                  '年齢':age,
                    '体重':weight,
                    'クラス':class_list,
                    'F数':F_count,
                    'L数':L_count,
                    '平均ST':mean_ST,
                    '勝率_全国':win_rate,
                    '2連率_全国':win2_rate,
                    '3連率_全国':win3_rate,
                    '勝率_当地':local_win_rate,
                    '2連率_当地':local_win2_rate,
                    '3連率_当地':local_win3_rate,
                    'モーターナンバー':motor_num,
                    'モーター2連率':motor_win2,
                    'モーター3連率':motor_win3,
                    'ボートナンバー':boat_num,
                    'ボート2連率':motor_win2,
                    'ボート3連率':motor_win3,
                    'コース':course
                    })
      time.sleep(1)
      #レース結果
      res = requests.get(result_url)
      res.encoding = res.apparent_encoding
      soup = BeautifulSoup(res.text, 'html.parser')

      rank_list = []
      rank = soup.find_all('td',class_="is-fBold")
      for i in rank[1:]:
        rank_list.append(i.get_text())
      rank_df = pd.DataFrame({"コース":rank_list,
                    "順位":course})
      rank_df["コース"] = rank_df['コース'].astype(int)
      #new_df['順位'] = rank_list
      new_df['日'] = p
      new_df['ラウンド'] = r

      new_df = pd.merge(new_df,rank_df,on="コース")
      df = pd.concat([df,new_df],axis=0)
      time.sleep(1)
    else:
      print('データがありません')

In [ ]:
df

,名前,年齢,体重,クラス,F数,L数,平均ST,勝率_全国,2連率_全国,3連率_全国,...,モーターナンバー,モーター2連率,モーター3連率,ボートナンバー,ボート2連率,ボート3連率,コース,日,ラウンド,順位
0,中嶋 誠一郎\n,51歳,51.5,A2,F0,L0,0.16,5.74,36.46,57.29,...,75\n,28.74,50.57\n,143,28.74,50.57\n,1,01,1,1
1,小林 孝弘\n,41歳,51.0,B1,F0,L0,0.15,3.95,18.92,35.14,...,73\n,41.67,60.42\n,152,41.67,60.42\n,2,01,1,3
2,高野 哲史\n,33歳,52.0,A1,F0,L0,0.14,6.78,54.40,67.20,...,65\n,29.03,47.31\n,159,29.03,47.31\n,3,01,1,2
3,松井 洪弥\n,29歳,52.0,A2,F0,L0,0.14,6.46,47.31,63.44,...,24\n,36.11,55.56\n,151,36.11,55.56\n,4,01,1,6
4,土井 祥伍\n,28歳,54.1,B1,F0,L0,0.20,4.48,25.49,40.20,...,17\n,38.46,56.73\n,111,38.46,56.73\n,5,01,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,中村 日向\n,24歳,53.0,A2,F0,L0,0.15,7.02,55.56,77.04,...,30\n,37.09,54.97\n,125,37.09,54.97\n,2,30,12,2
2,北中 元樹\n,43歳,54.7,A2,F0,L0,0.17,5.22,32.31,48.46,...,27\n,41.12,55.14\n,102,41.12,55.14\n,3,30,12,3
3,茶谷 信次\n,46歳,52.6,A2,F0,L0,0.20,5.58,38.68,60.38,...,74\n,36.13,58.82\n,123,36.13,58.82\n,4,30,12,6
4,河野 真也\n,33歳,52.0,B1,F1,L0,0.17,5.82,44.83,56.90,...,39\n,22.31,38.46\n,163,22.31,38.46\n,5,30,12,5


In [ ]:
df.to_csv('/content/drive/MyDrive/ボートレース予測/スクレイピング/１２月分データ修正版.csv',index='False')